In [16]:
!pip install scikit-learn
!pip install pandas
!pip install matplotlib
!pip install numpy
!pip install seaborn


  Using cached plotly-5.14.1-py2.py3-none-any.whl (15.3 MB)
ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [17]:
pip install plotly


  Using cached plotly-5.14.1-py2.py3-none-any.whl (15.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Mall Customers").getOrCreate()
df = spark.read.csv("Mall_Customers.csv", inferSchema=True,sep=",",header=True)
df.show()

+----------+------+---+-------------+--------------+
|CustomerID|Gender|Age|Annual_Income|Spending_Score|
+----------+------+---+-------------+--------------+
|         1|  Male| 19|           15|            39|
|         2|  Male| 21|           15|            81|
|         3|Female| 20|           16|             6|
|         4|Female| 23|           16|            77|
|         5|Female| 31|           17|            40|
|         6|Female| 22|           17|            76|
|         7|Female| 35|           18|             6|
|         8|Female| 23|           18|            94|
|         9|  Male| 64|           19|             3|
|        10|Female| 30|           19|            72|
|        11|  Male| 67|           19|            14|
|        12|Female| 35|           19|            99|
|        13|Female| 58|           20|            15|
|        14|Female| 24|           20|            77|
|        15|  Male| 37|           20|            13|
|        16|  Male| 22|           20|         

In [5]:
customers = df.select("Annual_Income","Spending_Score")
customers.show()

+-------------+--------------+
|Annual_Income|Spending_Score|
+-------------+--------------+
|           15|            39|
|           15|            81|
|           16|             6|
|           16|            77|
|           17|            40|
|           17|            76|
|           18|             6|
|           18|            94|
|           19|             3|
|           19|            72|
|           19|            14|
|           19|            99|
|           20|            15|
|           20|            77|
|           20|            13|
|           20|            79|
|           21|            35|
|           21|            66|
|           23|            29|
|           23|            98|
+-------------+--------------+
only showing top 20 rows



In [39]:
import plotly.express as px
import plotly.io as pio
fig = px.scatter(customers.toPandas(), y='Spending_Score', x='Annual_Income')
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
    width = 400
)
fig.show()
pio.renderers.default = 'iframe'

In [27]:
from pyspark.ml.feature import VectorAssembler
X = customers.columns
assembler = VectorAssembler(inputCols=X, outputCol="features")
assembled = assembler.transform(customers)
assembled.show()


+-------------+--------------+-----------+
|Annual_Income|Spending_Score|   features|
+-------------+--------------+-----------+
|           15|            39|[15.0,39.0]|
|           15|            81|[15.0,81.0]|
|           16|             6| [16.0,6.0]|
|           16|            77|[16.0,77.0]|
|           17|            40|[17.0,40.0]|
|           17|            76|[17.0,76.0]|
|           18|             6| [18.0,6.0]|
|           18|            94|[18.0,94.0]|
|           19|             3| [19.0,3.0]|
|           19|            72|[19.0,72.0]|
|           19|            14|[19.0,14.0]|
|           19|            99|[19.0,99.0]|
|           20|            15|[20.0,15.0]|
|           20|            77|[20.0,77.0]|
|           20|            13|[20.0,13.0]|
|           20|            79|[20.0,79.0]|
|           21|            35|[21.0,35.0]|
|           21|            66|[21.0,66.0]|
|           23|            29|[23.0,29.0]|
|           23|            98|[23.0,98.0]|
+----------

In [51]:
from pyspark.ml.clustering import KMeans 
k_means = KMeans(k=5,featuresCol='features')
fited = k_means.fit(assembled)
transformed = fited.transform(assembled)


In [52]:
transformed.select("features").show(2)

+-----------+
|   features|
+-----------+
|[15.0,39.0]|
|[15.0,81.0]|
+-----------+
only showing top 2 rows



In [53]:
from pyspark.ml.functions import vector_to_array
features_xy = transformed.withColumn('x',vector_to_array('features')[0])\
          .withColumn('y',vector_to_array('features')[1])

features_xy.select(['x','y','prediction']).show()


+----+----+----------+
|   x|   y|prediction|
+----+----+----------+
|15.0|39.0|         1|
|15.0|81.0|         3|
|16.0| 6.0|         1|
|16.0|77.0|         3|
|17.0|40.0|         1|
|17.0|76.0|         3|
|18.0| 6.0|         1|
|18.0|94.0|         3|
|19.0| 3.0|         1|
|19.0|72.0|         3|
|19.0|14.0|         1|
|19.0|99.0|         3|
|20.0|15.0|         1|
|20.0|77.0|         3|
|20.0|13.0|         1|
|20.0|79.0|         3|
|21.0|35.0|         1|
|21.0|66.0|         3|
|23.0|29.0|         1|
|23.0|98.0|         3|
+----+----+----------+
only showing top 20 rows



In [54]:
fig = px.scatter(features_xy.toPandas(),x='x',y='y', color='prediction')
fig.show()
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
    width = 800
)
pio.renderers.default = 'iframe'